In [45]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [2]:
import os

In [93]:
ROLES ={
   "1":{
       "name":"倫理エキスパート",
       "description":"倫理的な観点からチェックするエキスパート",
       "details":"倫理的な観点に関する知識が豊富で、文面の内容が倫理的な観点から問題ないかを評価できます。"
   },
   "2":{
       "name":"技術エキスパート",
       "description":"技術的な質問に答えることが得意なエキスパートです。",
       "details":"プログラミング、ソフトウェア開発、ITインフラストラクチャなどの技術分野に精通しており、専門的な質問に対応できます。"
   },
   "3":{
       "name":"ビジネスエキスパート",
       "description":"ビジネスに関する質問に答えることが得意なエキスパートです。",
       "details":"マーケティング、戦略、経営などのビジネス分野に精通しており、実践的なアドバイスを提供できます。"
   }
}

In [94]:
import operator
from typing import Annotated

from langchain_core.pydantic_v1 import BaseModel, Field

In [95]:
# 状態遷移を保持するクラス
class State(BaseModel):
    query: str = Field(
        description="ユーザーからの質問"
    )
    current_role: str = Field(
        default="",
        description="選定された回答ロール"
    )
    # add operator で、リストを結合するおかげですべてのメッセージが State に保持される
    messages: Annotated[list[str], operator.add] = Field(
        default=[], description="回答履歴"
    )
    judgement_result: bool = Field(
        default=False,
        description="品質チェックの結果"
    )
    judgement_reason: str = Field(
        default="",
        description="品質チェックの理由"
    )


In [96]:
# Azure OpenAI のインスタンスを準備
from langchain_openai import AzureChatOpenAI
from langchain_core.runnables import ConfigurableField
from azure.identity import ClientSecretCredential, get_bearer_token_provider

token_provider = get_bearer_token_provider(
    ClientSecretCredential(
        client_id=os.getenv("client_id"),
        client_secret=os.getenv("client_secret"),
        tenant_id=os.getenv("tenant_id")
    ),
    "https://cognitiveservices.azure.com/.default"
)
aoai = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_version="2025-02-01-preview",
    azure_ad_token_provider=token_provider,
    temperature=0
)
# 後からパラメータを変更できるように ConfigurableField でラップ
aoai = aoai.configurable_fields(max_tokens=ConfigurableField(id='max_tokens'))

In [97]:
aoai.invoke([{"role":"user","content":"Hello!"}])

AIMessage(content='Hi there! 😊 How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 9, 'total_tokens': 21, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-11-20', 'system_fingerprint': 'fp_3eed281ddb', 'id': 'chatcmpl-CDVfEHPQgYvLLCghQLmtivFolyuDh', 'service_tier': None, 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'

In [98]:
from typing import Any
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def selection_node(state: State) -> str:
    # まずは、state に保存されている現在のクエリを与える
    query = state.query
    # \n(改行) で ROLES の内容を連結して文字列にする（あとで LLM に渡すため）
    role_options = "\n".join([f"{k}:{v['name']} - {v['description']}" for k,v in ROLES.items()])
    # LLM に与えるプロンプトを作成
    prompt = ChatPromptTemplate.from_template(
        """あなたは優秀なアシスタントです。以下の質問に最も適したエキスパートを選んでください。

        質問: {query}

        選択肢:
        {role_options}
        
        回答は選択肢の番号(1,2,3)のどれかのみ返してください。
        """.strip()
    )
    # 選択肢の番号のみを返すように設定したいため、max_tokens を 1 に設定
    chain = prompt | aoai.with_config(configurable=dict(max_tokens=1)) | StrOutputParser()
    selected_role_num = chain.invoke({"query": query, "role_options": role_options}).strip()
    selected_role = ROLES[selected_role_num]["name"] 
    return {"current_role": selected_role}

In [99]:
type(ROLES['1'])
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [100]:
def answering_node(state: State) -> dict[str, Any]:
    query = state.query
    role = state.current_role
    # current role に基づいて LLM が回答するため、説明としては全ロールの説明を入れておく必要がある
    role_options = "\n".join([f"-{v['name']} - {v['description']}" for v in ROLES.values()])
    # 静的に変数として role_num を受け取っていれば、以下のように書けるが、今回は書籍に合わせる
    # role_detail = f"-{role['name']}: {role['description']}"
    prompt = ChatPromptTemplate.from_template(
        """あなたは {role} として回答してください。また回答の最初に自分がどのロールで回答しているかを明記してください。
        
        ロール定義
        {role_options}
        
        質問: {query}
        
        回答:
        """.strip()
    )
    chain = prompt | aoai | StrOutputParser()
    ans = chain.invoke({"query": query, "role": role, "role_options": role_options})
    return {"messages": [ans]}  # messages は list[str] なので、list にして返す -> 戻り値を受け取った側で append するため
    

In [101]:
class Judgement(BaseModel):
    reason: str = Field(
        description="品質チェックの理由",
        default=""
    )
    judge: bool = Field(
        description="品質チェックの結果",
        default=False
    )

def check_node(state: State) -> dict[str, Any]:
    query = state.query
    answer = state.messages[-1]  # 最新の回答を取得
    prompt = ChatPromptTemplate.from_template("""
                                              以下の回答をチェックし、品質に問題がある場合は、'False'、問題がない場合は 'True' を返してください。
                                              
                                                質問: {query}
                                                回答: {answer}
                                              
                                              """.strip())
    chain = prompt | aoai.with_structured_output(Judgement) 
    result:Judgement = chain.invoke({"query": query, "answer": answer})
    return {
        "judgement_reason": result.reason,
        "judgement_result": result.judge
        }

In [102]:
from langgraph.graph import StateGraph
workflow = StateGraph(State)

In [103]:
from langgraph.graph import END
workflow.add_node("selection", selection_node)
workflow.add_node("answering", answering_node)
workflow.add_node("check", check_node)

workflow.set_entry_point("selection")
workflow.add_edge("selection", "answering")
workflow.add_edge("answering", "check")
workflow.add_conditional_edges("check", lambda state: state.judgement_result, {True: END, False: "selection"})

compiled = workflow.compile()

In [105]:
init_state = State(query="医療業界において生成AIを用いたビジネスを生み出そうとした際に、どのようなものが考えられますか？データのプライバシーや倫理的な観点も考慮してください。")
result = compiled.invoke(init_state)

c:\Users\koishizu\AppData\Local\anaconda3\envs\langgraph_env\Lib\site-packages\langchain_openai\chat_models\base.py:1896: UserWarning: Received a Pydantic BaseModel V1 schema. This is not supported by method="json_schema". Please use method="function_calling" or specify schema via JSON Schema or Pydantic V2 BaseModel. Overriding to method="function_calling".
  warnings.warn(


In [106]:
from pprint import pprint
pprint(result['messages'])
# for m in result['messages']:
#     pprint(m)

['**ビジネスエキスパートとして回答します。**\n'
 '\n'
 '医療業界において生成AIを活用したビジネスを構築する際には、以下のようなアイデアが考えられます。ただし、データのプライバシーや倫理的な観点を十分に考慮する必要があります。\n'
 '\n'
 '---\n'
 '\n'
 '### 1. **患者向けのパーソナライズド健康アドバイザー**\n'
 '生成AIを活用して、患者の健康データ（例：電子カルテ、ウェアラブルデバイスのデータ）を分析し、個別化された健康アドバイスを提供するサービスを構築できます。たとえば、食事、運動、睡眠の改善提案や、慢性疾患の管理サポートなどが挙げられます。\n'
 '\n'
 '- **ビジネスメリット**: 健康意識の高まりに伴い、個別化されたサービスの需要が増加。\n'
 '- **倫理的配慮**: 患者データの匿名化、データ利用の透明性、AIのアドバイスが医療行為と誤解されないよう明確化。\n'
 '\n'
 '---\n'
 '\n'
 '### 2. **医療従事者向けの診断支援ツール**\n'
 '生成AIを用いて、医師や看護師が診断や治療計画を立てる際のサポートを行うツールを開発します。例えば、患者の症状や検査結果を入力すると、AIが考えられる診断候補や治療オプションを提示する仕組みです。\n'
 '\n'
 '- **ビジネスメリット**: 医療従事者の負担軽減、診断精度の向上。\n'
 '- **倫理的配慮**: AIの診断結果が医師の判断を完全に代替するものではないことを明確化。誤診リスクを最小化するための検証プロセスが必要。\n'
 '\n'
 '---\n'
 '\n'
 '### 3. **医療コンテンツ生成プラットフォーム**\n'
 '生成AIを活用して、患者向けの医療情報や教育コンテンツを自動生成するプラットフォームを提供します。たとえば、特定の病気に関する説明や治療法の概要を、患者の理解度に応じてカスタマイズして提供するサービスです。\n'
 '\n'
 '- **ビジネスメリット**: 医療情報へのアクセスを容易にし、患者の理解を深める。\n'
 '- **倫理的配慮**: 提供する情報の正確性を保証するため、医療専門家による監修が必要。誤情報の拡散を防ぐ仕組みを構築。\n'
 '\


マルチエージェントの構成にしているわけではないので、あくまで与えられた複数のエキスパートから一つを選んで回答を生成し、問題なければそこで回答になるというグラフに過ぎない